In [1]:
from keras.models import Sequential
from keras import layers
import numpy as np
from six.moves import range

/Users/kuoteng/.pyenv/versions/3.6.4/envs/NCKU-DSAI/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Parameters Config

In [2]:
class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'

In [3]:
DATA_SIZE = 60000
TRAIN_SIZE = 45000
DIGITS = 3
REVERSE = False
MAXLEN = DIGITS + DIGITS + 1
chars = '0123456789+-* '
RNN = layers.LSTM
HIDDEN_SIZE = 128
BATCH_SIZE = 128
EPOCH_SIZE = 2
LAYERS = 1

In [4]:
class CharacterTable(object):
    def __init__(self, chars):
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))
    
    def encode(self, C, num_rows):
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x
    
    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return "".join(self.indices_char[i] for i in x)

In [5]:
ctable = CharacterTable(chars)

In [6]:
ctable.indices_char

{0: ' ',
 1: '*',
 2: '+',
 3: '-',
 4: '0',
 5: '1',
 6: '2',
 7: '3',
 8: '4',
 9: '5',
 10: '6',
 11: '7',
 12: '8',
 13: '9'}

# Data Generation

In [7]:
%time
questions = []
expected = []
seen = set()
print('Generating data...')

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 9.06 µs
Generating data...


In [8]:
%time
while len(questions) < DATA_SIZE:
    f = lambda: int(''.join(np.random.choice(list('0123456789')) for i in range(np.random.randint(1, DIGITS + 1))))
    a, b = f(), f()
    if len(questions) % 3 == 1:
        q = '{}-{}'.format(a, b)
        query = q + ' ' * (MAXLEN - len(q))
        ans = str(a - b)
    elif len(questions) % 3 == 2:
        q = '{}+{}'.format(a, b)
        query = q + ' ' * (MAXLEN - len(q))
        ans = str(a + b)
    else:
        q = '{}*{}'.format(a, b)
        query = q + ' ' * (MAXLEN - len(q))
        ans = str(a * b)
    if q in seen:
        continue
    seen.add(q)
    ans += ' ' * (MAXLEN - len(ans))
    if REVERSE:
        query = query[::-1]
    questions.append(query)
    expected.append(ans)
print('Total addition questions:', len(questions))

CPU times: user 6 µs, sys: 1e+03 ns, total: 7 µs
Wall time: 13.8 µs
Total addition questions: 60000


In [9]:
print(questions[:5], expected[:5])

['630*8  ', '3-294  ', '54+89  ', '13*138 ', '348-40 '] ['5040   ', '-291   ', '143    ', '1794   ', '308    ']


# Processing

In [10]:
print('Vectorization... (to the one-hot encoding)')
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(expected), MAXLEN, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, MAXLEN)

Vectorization... (to the one-hot encoding)


In [11]:
indices = np.arange(len(y))
np.random.shuffle(indices)
print(indices)
x = x[indices]
y = y[indices]

# train_test_split
train_x = x[:TRAIN_SIZE]
train_y = y[:TRAIN_SIZE]
test_x = x[TRAIN_SIZE:]
test_y = y[TRAIN_SIZE:]

print('Training Data:')
print(train_x.shape)
print(train_y.shape)

split_at = len(train_x) - len(train_x) // 10
print('split_at', split_at)
(x_train, x_val) = train_x[:split_at], train_x[split_at:]
(y_train, y_val) = train_y[:split_at], train_y[split_at:]

print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)

print('Testing Data:')
print(test_x.shape)
print(test_y.shape)


[55863 26383 45919 ... 19068 56564 43708]
Training Data:
(45000, 7, 14)
(45000, 7, 14)
split_at 40500
Training Data:
(40500, 7, 14)
(40500, 7, 14)
Validation Data:
(4500, 7, 14)
(4500, 7, 14)
Testing Data:
(15000, 7, 14)
(15000, 7, 14)


In [12]:
print("input: ", x_train[:3], '\n\n', "label: ", y_train[:3])

input:  [[[False False False False False False False False False False False
    True False False]
  [False False False False False False False False False False False
    True False False]
  [False  True False False False False False False False False False
   False False False]
  [False False False False False  True False False False False False
   False False False]
  [False False False False False False False False False False False
    True False False]
  [ True False False False False False False False False False False
   False False False]
  [ True False False False False False False False False False False
   False False False]]

 [[False False False False False False False False  True False False
   False False False]
  [False False False False False False False False False  True False
   False False False]
  [False False False  True False False False False False False False
   False False False]
  [False False False False False False False False False False False
    True Fa

# Build Model

In [13]:
print('Build model...')
model = Sequential()
model.add(RNN(HIDDEN_SIZE, input_shape=(MAXLEN, len(chars))))
model.add(layers.RepeatVector(MAXLEN))
for _ in range(LAYERS):
    model.add(RNN(HIDDEN_SIZE, return_sequences=True))

model.add(layers.TimeDistributed(layers.Dense(len(chars))))
model.add(layers.Activation('softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               73216     
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 7, 128)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 7, 128)            131584    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 7, 14)             1806      
_________________________________________________________________
activation_1 (Activation)    (None, 7, 14)             0         
Total params: 206,606
Trainable params: 206,606
Non-trainable params: 0
_________________________________________________________________


# Training

In [14]:
% time
for loop in range(50):
    print()
    print('-' * 50)
    print('Train Loop Num:', loop)
    model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=EPOCH_SIZE,
              validation_data=(x_val, y_val),
              shuffle=True)
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.2 µs

--------------------------------------------------
Train Loop Num: 0
Train on 40500 samples, validate on 4500 samples
Epoch 1/2
40500/40500 [==============================] - 14s 339us/step - loss: 1.3402 - acc: 0.5471 - val_loss: 1.1656 - val_acc: 0.5811
Epoch 2/2
40500/40500 [==============================] - 12s 288us/step - loss: 1.1394 - acc: 0.5882 - val_loss: 1.1211 - val_acc: 0.5900
Q 640+562 T 1202    ☒ 100    
Q 67+703  T 770     ☒ 109    
Q 529*563 T 297827  ☒ 123555 
Q 85+164  T 249     ☒ 109    
Q 409-875 T -466    ☒ -10    
Q 732+54  T 786     ☒ 197    
Q 88+842  T 930     ☒ 199    
Q 826-884 T -58     ☒ -26    
Q 992-16  T 976     ☒ 109    
Q 0+197   T 197     ☒ 107    

--------------------------------------------------
Train Loop Num: 1
Train on 40500 samples, validate on 4500 samples
Epoch 1/2
40500/40500 [==============================] - 12s 292us/step - loss: 1.0975 - acc: 0.5989 - val_loss: 1.0802 - v

40500/40500 [==============================] - 12s 297us/step - loss: 0.5483 - acc: 0.7852 - val_loss: 0.5824 - val_acc: 0.7650
Q 4+810   T 814     ☒ 824    
Q 41+193  T 234     ☒ 230    
Q 76+152  T 228     ☒ 234    
Q 5+358   T 363     ☑ 363    
Q 95-432  T -337    ☒ -347   
Q 684+9   T 693     ☒ 763    
Q 247-28  T 219     ☒ 245    
Q 24*28   T 672     ☒ 654    
Q 64+30   T 94      ☒ 92     
Q 65*6    T 390     ☑ 390    

--------------------------------------------------
Train Loop Num: 11
Train on 40500 samples, validate on 4500 samples
Epoch 1/2
40500/40500 [==============================] - 12s 295us/step - loss: 0.5367 - acc: 0.7901 - val_loss: 0.5738 - val_acc: 0.7691
Epoch 2/2
40500/40500 [==============================] - 11s 279us/step - loss: 0.5258 - acc: 0.7954 - val_loss: 0.5669 - val_acc: 0.7735
Q 173+92  T 265     ☑ 265    
Q 31+55   T 86      ☑ 86     
Q 405+171 T 576     ☒ 544    
Q 6-612   T -606    ☒ -610   
Q 857*36  T 30852   ☒ 29678  
Q 411-40  T 371     ☒ 399 

40500/40500 [==============================] - 12s 297us/step - loss: 0.3178 - acc: 0.8846 - val_loss: 0.3977 - val_acc: 0.8483
Epoch 2/2
40500/40500 [==============================] - 12s 290us/step - loss: 0.3149 - acc: 0.8859 - val_loss: 0.3859 - val_acc: 0.8526
Q 3-76    T -73     ☑ -73    
Q 95*82   T 7790    ☒ 7780   
Q 7+340   T 347     ☑ 347    
Q 454*15  T 6810    ☒ 6680   
Q 87-5    T 82      ☑ 82     
Q 88+18   T 106     ☑ 106    
Q 3*417   T 1251    ☒ 1291   
Q 641*33  T 21153   ☒ 20191  
Q 68+92   T 160     ☑ 160    
Q 33*68   T 2244    ☒ 2294   

--------------------------------------------------
Train Loop Num: 22
Train on 40500 samples, validate on 4500 samples
Epoch 1/2
40500/40500 [==============================] - 12s 295us/step - loss: 0.2998 - acc: 0.8927 - val_loss: 0.3821 - val_acc: 0.8556
Epoch 2/2
40500/40500 [==============================] - 12s 300us/step - loss: 0.2947 - acc: 0.8942 - val_loss: 0.3774 - val_acc: 0.8552
Q 354-65  T 289     ☒ 299    
Q 327+50

40500/40500 [==============================] - 12s 291us/step - loss: 0.2037 - acc: 0.9294 - val_loss: 0.3461 - val_acc: 0.8751
Epoch 2/2
40500/40500 [==============================] - 13s 318us/step - loss: 0.2006 - acc: 0.9302 - val_loss: 0.3319 - val_acc: 0.8797
Q 219+5   T 224     ☑ 224    
Q 343*570 T 195510  ☒ 198990 
Q 5*425   T 2125    ☒ 2175   
Q 179-8   T 171     ☑ 171    
Q 63+2    T 65      ☑ 65     
Q 831*56  T 46536   ☒ 49976  
Q 481+934 T 1415    ☒ 1425   
Q 670+5   T 675     ☑ 675    
Q 9*506   T 4554    ☒ 4034   
Q 29+12   T 41      ☑ 41     

--------------------------------------------------
Train Loop Num: 33
Train on 40500 samples, validate on 4500 samples
Epoch 1/2
40500/40500 [==============================] - 14s 347us/step - loss: 0.2006 - acc: 0.9304 - val_loss: 0.3417 - val_acc: 0.8764
Epoch 2/2
40500/40500 [==============================] - 15s 359us/step - loss: 0.1972 - acc: 0.9313 - val_loss: 0.3340 - val_acc: 0.8785
Q 13+17   T 30      ☑ 30     
Q 103+43

40500/40500 [==============================] - 17s 426us/step - loss: 0.1448 - acc: 0.9517 - val_loss: 0.3519 - val_acc: 0.8827
Epoch 2/2
40500/40500 [==============================] - 14s 345us/step - loss: 0.1439 - acc: 0.9518 - val_loss: 0.3593 - val_acc: 0.8808
Q 668-589 T 79      ☒ 99     
Q 319+0   T 319     ☑ 319    
Q 359*43  T 15437   ☒ 16657  
Q 406-6   T 400     ☒ 302    
Q 28-471  T -443    ☑ -443   
Q 0+90    T 90      ☑ 90     
Q 44-54   T -10     ☑ -10    
Q 767-0   T 767     ☑ 767    
Q 858+2   T 860     ☒ 850    
Q 3+922   T 925     ☑ 925    

--------------------------------------------------
Train Loop Num: 44
Train on 40500 samples, validate on 4500 samples
Epoch 1/2
40500/40500 [==============================] - 16s 389us/step - loss: 0.1508 - acc: 0.9493 - val_loss: 0.3598 - val_acc: 0.8821
Epoch 2/2
40500/40500 [==============================] - 13s 331us/step - loss: 0.1399 - acc: 0.9534 - val_loss: 0.3728 - val_acc: 0.8801
Q 642+1   T 643     ☑ 643    
Q 735-81

# Testing

In [15]:
print("MSG : Prediction")
print("-" * 50)
right = 0
preds = model.predict_classes(test_x, verbose=0)
for i in range(len(preds)):
    q = ctable.decode(test_x[i])
    correct = ctable.decode(test_y[i])
    guess = ctable.decode(preds[i], calc_argmax=False)
    print('Q', q[::-1] if REVERSE else q, end=' ')
    print('T', correct, end=' ')
    if correct == guess:
        print(colors.ok + '☑' + colors.close, end=' ')
        right += 1
    else:
        print(colors.fail + '☒' + colors.close, end=' ')
    print(guess)
print("MSG : Accuracy is {}".format(right / len(preds)))

MSG : Prediction
--------------------------------------------------
Q 244*28  T 6832    ☒ 7352   
Q 587*52  T 30524   ☒ 20674  
Q 23+195  T 218     ☑ 218    
Q 850*139 T 118150  ☒ 114850 
Q 467-8   T 459     ☑ 459    
Q 7-162   T -155    ☑ -155   
Q 280*182 T 50960   ☒ 51760  
Q 640*61  T 39040   ☒ 3904   
Q 960+493 T 1453    ☒ 1443   
Q 22-53   T -31     ☑ -31    
Q 10*3    T 30      ☑ 30     
Q 39+73   T 112     ☑ 112    
Q 131-4   T 127     ☒ 137    
Q 26*8    T 208     ☑ 208    
Q 2*360   T 720     ☑ 720    
Q 77*980  T 75460   ☒ 75660  
Q 136-241 T -105    ☒ -197   
Q 57*796  T 45372   ☒ 44672  
Q 651+71  T 722     ☑ 722    
Q 796-3   T 793     ☑ 793    
Q 29-9    T 20      ☑ 20     
Q 397-56  T 341     ☒ 331    
Q 581-724 T -143    ☒ -153   
Q 246-179 T 67      ☒ -37    
Q 47-91   T -44     ☑ -44    
Q 56+422  T 478     ☑ 478    
Q 955+0   T 955     ☑ 955    
Q 89-274  T -185    ☑ -185   
Q 886*686 T 607796  ☒ 622464 
Q 630+68  T 698     ☒ 688    
Q 69*87   T 6003    ☒ 6163   
Q 

Q 92*88   T 8096    ☒ 8114   
Q 887-55  T 832     ☑ 832    
Q 651+41  T 692     ☒ 782    
Q 857+24  T 881     ☑ 881    
Q 0*46    T 0       ☑ 0      
Q 509-422 T 87      ☒ 97     
Q 69+75   T 144     ☑ 144    
Q 65*298  T 19370   ☒ 19710  
Q 200*65  T 13000   ☒ 13200  
Q 89*42   T 3738    ☒ 3718   
Q 89-289  T -200    ☒ -190   
Q 978+564 T 1542    ☒ 1552   
Q 30*743  T 22290   ☒ 21070  
Q 1+2     T 3       ☑ 3      
Q 31+62   T 93      ☑ 93     
Q 4*88    T 352     ☒ 342    
Q 80-579  T -499    ☑ -499   
Q 75+3    T 78      ☑ 78     
Q 0+671   T 671     ☑ 671    
Q 3+645   T 648     ☒ 658    
Q 30-47   T -17     ☑ -17    
Q 254*41  T 10414   ☒ 10604  
Q 632+282 T 914     ☒ 904    
Q 53*558  T 29574   ☒ 29624  
Q 100-497 T -397    ☒ -481   
Q 67*500  T 33500   ☒ 34400  
Q 138-11  T 127     ☑ 127    
Q 9*191   T 1719    ☑ 1719   
Q 9*895   T 8055    ☒ 7925   
Q 5+236   T 241     ☑ 241    
Q 3*948   T 2844    ☒ 2804   
Q 734+116 T 850     ☒ 800    
Q 5+205   T 210     ☑ 210    
Q 1+447   

Q 464*38  T 17632   ☒ 17512  
Q 908+6   T 914     ☑ 914    
Q 679*95  T 64505   ☒ 64805  
Q 648*654 T 423792  ☒ 426052 
Q 8*32    T 256     ☒ 246    
Q 965*0   T 0       ☑ 0      
Q 77-926  T -849    ☑ -849   
Q 1-789   T -788    ☑ -788   
Q 829-98  T 731     ☒ 721    
Q 425-64  T 361     ☒ 351    
Q 144+1   T 145     ☑ 145    
Q 192*21  T 4032    ☒ 3832   
Q 580+4   T 584     ☑ 584    
Q 702-1   T 701     ☑ 701    
Q 11+571  T 582     ☒ 572    
Q 123-149 T -26     ☒ -3     
Q 623*56  T 34888   ☒ 34518  
Q 456+664 T 1120    ☑ 1120   
Q 11-78   T -67     ☑ -67    
Q 27+20   T 47      ☑ 47     
Q 27+287  T 314     ☑ 314    
Q 96-5    T 91      ☑ 91     
Q 8+228   T 236     ☑ 236    
Q 645-45  T 600     ☑ 600    
Q 45-65   T -20     ☑ -20    
Q 973-78  T 895     ☑ 895    
Q 6*834   T 5004    ☒ 4004   
Q 90*824  T 74160   ☒ 75780  
Q 975*693 T 675675  ☒ 682005 
Q 6*18    T 108     ☑ 108    
Q 7-241   T -234    ☒ -224   
Q 992+270 T 1262    ☑ 1262   
Q 29-739  T -710    ☑ -710   
Q 5*893   

Q 4+68    T 72      ☑ 72     
Q 2-30    T -28     ☑ -28    
Q 94-965  T -871    ☑ -871   
Q 569-67  T 502     ☑ 502    
Q 917+65  T 982     ☒ 972    
Q 470-70  T 400     ☑ 400    
Q 63+96   T 159     ☑ 159    
Q 44-135  T -91     ☑ -91    
Q 79-58   T 21      ☑ 21     
Q 627+678 T 1305    ☒ 1295   
Q 787-6   T 781     ☑ 781    
Q 615-734 T -119    ☒ -129   
Q 592+868 T 1460    ☒ 1450   
Q 368-141 T 227     ☒ 215    
Q 784+96  T 880     ☒ 870    
Q 119*3   T 357     ☒ 317    
Q 9*303   T 2727    ☒ 2747   
Q 4+59    T 63      ☑ 63     
Q 347+538 T 885     ☒ 875    
Q 567*3   T 1701    ☒ 1761   
Q 21-35   T -14     ☒ -24    
Q 555+715 T 1270    ☒ 1280   
Q 1-323   T -322    ☑ -322   
Q 21*741  T 15561   ☒ 15881  
Q 831-37  T 794     ☒ 894    
Q 1*453   T 453     ☑ 453    
Q 387-94  T 293     ☒ 283    
Q 338+850 T 1188    ☒ 1288   
Q 91*338  T 30758   ☒ 31858  
Q 726+4   T 730     ☑ 730    
Q 911-9   T 902     ☑ 902    
Q 33*78   T 2574    ☒ 2554   
Q 430+7   T 437     ☑ 437    
Q 289+7   

Q 499+56  T 555     ☒ 545    
Q 16-3    T 13      ☑ 13     
Q 400*251 T 100400  ☒ 123200 
Q 15-59   T -44     ☑ -44    
Q 0-962   T -962    ☒ -952   
Q 134-226 T -92     ☒ -22    
Q 462+6   T 468     ☑ 468    
Q 408*18  T 7344    ☒ 8588   
Q 706-898 T -192    ☒ -288   
Q 26-59   T -33     ☑ -33    
Q 985-67  T 918     ☒ 912    
Q 25*17   T 425     ☒ 455    
Q 5-718   T -713    ☑ -713   
Q 731-6   T 725     ☑ 725    
Q 87-0    T 87      ☑ 87     
Q 112+9   T 121     ☑ 121    
Q 2-37    T -35     ☑ -35    
Q 414-7   T 407     ☑ 407    
Q 270+980 T 1250    ☒ 1350   
Q 66*1    T 66      ☑ 66     
Q 535*6   T 3210    ☒ 3190   
Q 132*755 T 99660   ☒ 97420  
Q 221*3   T 663     ☑ 663    
Q 152*737 T 112024  ☒ 105581 
Q 6*762   T 4572    ☒ 4492   
Q 45*101  T 4545    ☒ 4565   
Q 127-5   T 122     ☒ 132    
Q 443-1   T 442     ☑ 442    
Q 481*343 T 164983  ☒ 166243 
Q 87*140  T 12180   ☒ 12080  
Q 8+383   T 391     ☑ 391    
Q 304+21  T 325     ☒ 215    
Q 658*38  T 25004   ☒ 24644  
Q 0-465   

...

Q 915*25  T 22875   ☒ 22755  
Q 32*73   T 2336    ☒ 2356   
Q 33+912  T 945     ☑ 945    
Q 59-890  T -831    ☒ -811   
Q 863*38  T 32794   ☒ 32084  
Q 330-3   T 327     ☑ 327    
Q 270*2   T 540     ☑ 540    
Q 89+64   T 153     ☑ 153    
Q 74*60   T 4440    ☒ 4460   
Q 42-37   T 5       ☒ 15     
Q 15-131  T -116    ☒ -124   
Q 19*64   T 1216    ☒ 1186   
Q 436-774 T -338    ☑ -338   
Q 18+7    T 25      ☑ 25     
Q 84*916  T 76944   ☒ 77054  
Q 311-91  T 220     ☑ 220    
Q 3-215   T -212    ☑ -212   
Q 773-377 T 396     ☒ 406    
Q 753+585 T 1338    ☒ 1458   
Q 77-910  T -833    ☑ -833   
Q 48-3    T 45      ☑ 45     
Q 322*86  T 27692   ☒ 28112  
Q 514-5   T 509     ☑ 509    
Q 48-33   T 15      ☑ 15     
Q 553*574 T 317422  ☒ 316282 
Q 25+8    T 33      ☑ 33     
Q 7*790   T 5530    ☒ 5330   
Q 72*0    T 0       ☑ 0      
Q 162*99  T 16038   ☒ 15038  
Q 1-356   T -355    ☒ -353   
Q 794*905 T 718570  ☒ 733880 
Q 86-24   T 62      ☑ 62     
Q 392-501 T -109    ☒ -13    
Q 475+757 

Q 112*394 T 44128   ☒ 42398  
Q 912+8   T 920     ☑ 920    
Q 65*948  T 61620   ☒ 59764  
Q 539+835 T 1374    ☒ 1354   
Q 6*176   T 1056    ☑ 1056   
Q 79*869  T 68651   ☒ 67551  
Q 15-0    T 15      ☑ 15     
Q 464+557 T 1021    ☒ 1011   
Q 57-6    T 51      ☑ 51     
Q 55+319  T 374     ☑ 374    
Q 445+0   T 445     ☑ 445    
Q 26-92   T -66     ☑ -66    
Q 5*261   T 1305    ☒ 1385   
Q 9+223   T 232     ☑ 232    
Q 13-349  T -336    ☒ -334   
Q 29+592  T 621     ☑ 621    
Q 791+680 T 1471    ☑ 1471   
Q 61-46   T 15      ☑ 15     
Q 21+540  T 561     ☑ 561    
Q 489-27  T 462     ☑ 462    
Q 586+9   T 595     ☒ 585    
Q 279*2   T 558     ☒ 538    
Q 448*893 T 400064  ☒ 413356 
Q 8*869   T 6952    ☒ 7052   
Q 8*852   T 6816    ☒ 6844   
Q 41-68   T -27     ☑ -27    
Q 697+84  T 781     ☒ 771    
Q 996-3   T 993     ☑ 993    
Q 861-7   T 854     ☑ 854    
Q 795+61  T 856     ☒ 944    
Q 15*57   T 855     ☒ 865    
Q 183-9   T 174     ☑ 174    
Q 3*625   T 1875    ☒ 1865   
Q 12*3    

Q 60+71   T 131     ☑ 131    
Q 5+324   T 329     ☑ 329    
Q 123*984 T 121032  ☒ 124858 
Q 834-6   T 828     ☑ 828    
Q 877+864 T 1741    ☒ 1731   
Q 971+797 T 1768    ☒ 1748   
Q 33+99   T 132     ☑ 132    
Q 99-526  T -427    ☑ -427   
Q 782*3   T 2346    ☑ 2346   
Q 2+745   T 747     ☑ 747    
Q 855-5   T 850     ☑ 850    
Q 299*23  T 6877    ☒ 6723   
Q 6-561   T -555    ☑ -555   
Q 28*864  T 24192   ☒ 23492  
Q 211*41  T 8651    ☒ 8971   
Q 2+246   T 248     ☒ 258    
Q 3*596   T 1788    ☒ 1798   
Q 89+994  T 1083    ☒ 1093   
Q 2*116   T 232     ☑ 232    
Q 85-799  T -714    ☒ -724   
Q 83-11   T 72      ☑ 72     
Q 40*692  T 27680   ☒ 29680  
Q 76-5    T 71      ☑ 71     
Q 62*57   T 3534    ☒ 3474   
Q 693+35  T 728     ☒ 738    
Q 536*679 T 363944  ☒ 346664 
Q 4+572   T 576     ☑ 576    
Q 201*8   T 1608    ☑ 1608   
Q 586-66  T 520     ☑ 520    
Q 860-9   T 851     ☑ 851    
Q 149-902 T -753    ☒ -755   
Q 342+761 T 1103    ☒ 1105   
Q 861*70  T 60270   ☒ 60870  
Q 202*144 

Q 768*953 T 731904  ☒ 735574 
Q 69-642  T -573    ☒ -575   
Q 531*271 T 143901  ☒ 145241 
Q 13*104  T 1352    ☒ 1632   
Q 0*88    T 0       ☑ 0      
Q 91*764  T 69524   ☒ 69174  
Q 455-0   T 455     ☑ 455    
Q 92*621  T 57132   ☒ 56852  
Q 668+74  T 742     ☒ 732    
Q 50-789  T -739    ☒ -749   
Q 42+46   T 88      ☑ 88     
Q 79-62   T 17      ☑ 17     
Q 2+118   T 120     ☒ 122    
Q 302*386 T 116572  ☒ 112892 
Q 96+787  T 883     ☒ 875    
Q 90*41   T 3690    ☒ 3650   
Q 27*31   T 837     ☑ 837    
Q 749*828 T 620172  ☒ 610452 
Q 41-894  T -853    ☒ -831   
Q 427+50  T 477     ☑ 477    
Q 7*440   T 3080    ☒ 3060   
Q 120-949 T -829    ☑ -829   
Q 9*844   T 7596    ☒ 7686   
Q 33+0    T 33      ☑ 33     
Q 94+4    T 98      ☑ 98     
Q 26*9    T 234     ☒ 224    
Q 919-946 T -27     ☒ -37    
Q 233+0   T 233     ☑ 233    
Q 829*26  T 21554   ☒ 21154  
Q 663+12  T 675     ☑ 675    
Q 782-123 T 659     ☑ 659    
Q 4+205   T 209     ☑ 209    
Q 48*723  T 34704   ☒ 34274  
Q 867+892 

Q 290+57  T 347     ☑ 347    
Q 62+879  T 941     ☑ 941    
Q 701+6   T 707     ☑ 707    
Q 84+352  T 436     ☑ 436    
Q 31*725  T 22475   ☒ 21775  
Q 87*28   T 2436    ☒ 2356   
Q 65*570  T 37050   ☒ 35050  
Q 938*780 T 731640  ☒ 755360 
Q 709*4   T 2836    ☑ 2836   
Q 71*236  T 16756   ☒ 15436  
Q 784*463 T 362992  ☒ 383632 
Q 4-58    T -54     ☑ -54    
Q 0*370   T 0       ☑ 0      
Q 470+131 T 601     ☒ 621    
Q 469+5   T 474     ☑ 474    
Q 39+56   T 95      ☑ 95     
Q 10+6    T 16      ☑ 16     
Q 436-10  T 426     ☑ 426    
Q 30*130  T 3900    ☒ 4500   
Q 269*2   T 538     ☑ 538    
Q 4-644   T -640    ☑ -640   
Q 77*12   T 924     ☒ 944    
Q 15-11   T 4       ☑ 4      
Q 445-404 T 41      ☒ 15     
Q 304-0   T 304     ☑ 304    
Q 366*98  T 35868   ☒ 36448  
Q 1-44    T -43     ☑ -43    
Q 93-22   T 71      ☑ 71     
Q 845+65  T 910     ☑ 910    
Q 295+983 T 1278    ☑ 1278   
Q 9*552   T 4968    ☒ 4808   
Q 50*28   T 1400    ☑ 1400   
Q 440+7   T 447     ☑ 447    
Q 621-25  

In [16]:
# print("MSG : Prediction")
# test_x = ["555+275", "860+7  ", "340+29 "]
# test_y = ["830 ", "867 ", "369 "]
# x = np.zeros((len(test_x), MAXLEN, len(chars)), dtype=np.bool)
# y = np.zeros((len(test_y), DIGITS + 1, len(chars)), dtype=np.bool)
# for j, (i, c) in enumerate(zip(test_x, test_y)):
#     x[j] = ctable.encode(i, MAXLEN)
#     y[j] = ctable.encode(c, DIGITS + 1)